# Compute projections
Best options is to run `iss project-round` command after each acquisition:

`iss project-round -p $DATAPATH -n $PREFIX`

where `$DATAPATH` is the path to the data relative to raw data root and `$PREFIX` is the file name prefix and directory name of the round, e.g. `genes_round_1_1`.


# Import requirements

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import iss_preprocess as iss
import numpy as np
import matplotlib.pyplot as plt
import yaml
from flexiznam.config import PARAMETERS
from pathlib import Path
from itertools import cycle
from matplotlib.animation import FuncAnimation


# Set options and path

In [ ]:
data_path = "becalia_rabies_barseq//BRAC6780.3b/chamber_19/"

processed_path = Path(PARAMETERS['data_root']['processed'])
# metadata = iss.io.load_metadata(data_path)

ops = iss.config.DEFAULT_OPS.copy()


In [ ]:
import os
os.listdir(processed_path / data_path)

In [ ]:
ops['camera_order'] = [1 ,3,4,2]
for cam, ch in enumerate(np.argsort(ops['camera_order'])):
    print(ch, cam)

In [ ]:
shift_right, shift_down, tile_shape = iss.pipeline.register_adjacent_tiles(
    data_path, ref_coors=ops['ref_tile'], prefix='mCherry_1')

In [ ]:
prefix='mCherry_1'
roi_dims = iss.pipeline.save_roi_dimensions(data_path, prefix)

In [ ]:
import cv2
fig = plt.figure(figsize=(20,20))

for roi in np.arange(8):
    fig.clear()
    print(f'Doing roi {roi+1}')
    print('   ... stitching', flush=True)
    stack = iss.pipeline.stitch_tiles(
        data_path,
        prefix,
        shift_right,
        shift_down,
        roi=roi + 1,
        suffix="fstack",
        ich=1,
        correct_illumination=False,
    )
    print('   ... downsampling', flush=True)
    downsampled_size = (np.array(stack.shape) / 20).astype('int')
    downsampled = cv2.resize(stack, downsampled_size[::-1], interpolation= cv2.INTER_LINEAR)
    print('   ... plotting', flush=True)
    plt.imshow(stack[::10, ::10], vmax=150)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.axis('off')
    print('   ... saving', flush=True)
    fig.savefig(processed_path / data_path / f'mcherry_overview_roi{roi + 1}.png', dpi=600)

# Montage

A montage can be generated from the overviews using ImageMagick:

```
ml ImageMagick
cd MOUSE_DIRECTORY
magick montage chamber_19/mcheery_overview*.png -geometry 2000x -tile 4x2 chamge_19_mCherry.png
```

The 2000x sets the number of pixels of each image in the montage (in x) and  4x2 the tile
arangment.